# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

import warnings
warnings.filterwarnings("ignore")

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("output_data/clean_city_data.csv")
city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,puerto ayora,-0.7393,-90.3518,68.56,96,93,10.00,EC,1.633667e+09
1,saldanha,-33.0117,17.9442,57.25,76,59,14.67,ZA,1.633667e+09
2,georgetown,5.4112,100.3354,86.90,80,20,1.01,MY,1.633666e+09
3,khatanga,71.9667,102.5000,21.87,95,100,3.94,RU,1.633667e+09
4,saint george,37.1041,-113.5841,72.95,48,1,4.00,US,1.633666e+09
...,...,...,...,...,...,...,...,...,...
555,agua branca,-5.8922,-42.6361,80.62,48,93,5.26,BR,1.633667e+09
556,santa helena de goias,-17.8136,-50.5969,75.79,68,86,7.72,BR,1.633667e+09
557,aginskoye,51.1000,114.5300,46.44,61,100,6.29,RU,1.633667e+09
558,zhezkazgan,47.8043,67.7144,38.80,56,1,13.15,KZ,1.633667e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = city_data[["Lat","Lng"]].astype(float)
humidity = city_data["Humidity"]
humidity.max()

100

In [4]:
fig_layout = {'width': '800px',
              'height': '400px',
              'border': '1px solid black',
              'padding': '1px',
              'margin': '0 auto 0 auto'
             }
fig = gmaps.figure(layout=fig_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=humidity.max(), point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Cloudiness Less than 5.
# Humidity Less than 50.
hotel_df = city_data.loc[(city_data['Max Temp']<80)&(city_data['Max Temp']>70)&(city_data['Wind Speed']<10)&(city_data['Cloudiness']<5)&(city_data['Humidity']<50),:]
hotel_df.index

Int64Index([4, 148, 190, 224, 236], dtype='int64')

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df['Hotel Name']=""

In [7]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {
          'radius':5000,
          'types':'hotel',
          'key':g_key,    
}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location']=f'{lat},{lng}'
    name = requests.get(base_url, params = params).json()
    try:
        hotel_df.loc[index,'Hotel Name'] = name['results'][1]['name']
        print(f"Searching for hotel in city {row['City']}")
    except:
        print('Missing result...skipping.')

Searching for hotel in city saint george
Searching for hotel in city arlit
Searching for hotel in city riyadh
Searching for hotel in city jalu
Searching for hotel in city matamoros


In [8]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,saint george,37.1041,-113.5841,72.95,48,1,4.00,US,1.633666e+09,Best Western Plus Abbey Inn
148,arlit,18.7369,7.3853,79.16,18,4,8.39,NE,1.633667e+09,Station Total
190,riyadh,24.6877,46.7219,77.14,36,0,3.04,SA,1.633667e+09,"InterContinental Riyadh, an IHG Hotel"
224,jalu,29.0331,21.5482,70.11,33,0,7.38,LY,1.633667e+09,fantry Kitchens
236,matamoros,25.5333,-103.2500,78.12,31,0,9.10,MX,1.633667e+09,Sepomex


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…